### Problem:

1. Write a function that generates one period of Gold code sequence for a given PRN number.

In [1]:
from typing import List
import numpy as np

def generate_mls(
    register_length: int, feedback_taps: List[int], output_taps: List[int]
):
    """Generates Maximum Length Sequence (length-(2**N - 1) binary sequence) for the given feedback and output taps.

    Parameters
    ----------
    `register_length` : number of bits to use in the feedback register
    `feedback_taps` : list of integer feedback tap indices
    `output_taps` : list of integer output tap indices

    Returns
    -------
    output : NDArray of shape (2**register_length - 1,)
        the code sequence
    """
    shift_register = np.ones((register_length,), dtype=np.int8)
    seq = np.zeros((2**register_length - 1,), dtype=np.int8)
    for i in range(2**register_length - 1):
        seq[i] = np.sum(shift_register[output_taps]) % 2
        first = np.sum(shift_register[feedback_taps]) % 2
        shift_register[1:] = shift_register[:-1]
        shift_register[0] = first
    return seq


CARRIER_FREQ: int = 1575420000
CODE_RATE: int = 1023000
CODE_LENGTH: int = 1023
DATA_SYMBOL_RATE: int = 50

"""
(svid, prn, ca_phase_select, x2_phase_select, ca_code_delay, p_code_delay, first_10_chips_ca, first_12_chips_p)
Tuple struct to store data from Table 3-I of the IS-GPS 200 
specification, which contains code phase assignment information for GPS L1 signal.

`ca_phase_select` is a 2-tuple in this structure.

`first_12_chips_p`, `first_10_chips_ca` are represented in octal,
which is why they are listed as strings here.

Note that SVID and PRN numbers differ only for SVIDs 65-69.
"""
CODE_PHASE_ASSIGNMENTS = {
    1: (1, 1, (2, 6), 1, 5, 1, "1440", "4444"),
    2: (2, 2, (3, 7), 2, 6, 2, "1620", "4000"),
    3: (3, 3, (4, 8), 3, 7, 3, "1710", "4333"),
    4: (4, 4, (5, 9), 4, 8, 4, "1744", "4377"),
    5: (5, 5, (1, 9), 5, 17, 5, "1133", "4355"),
    6: (6, 6, (2, 10), 6, 18, 6, "1455", "4344"),
    7: (7, 7, (1, 8), 7, 139, 7, "1131", "4340"),
    8: (8, 8, (2, 9), 8, 140, 8, "1454", "4342"),
    9: (9, 9, (3, 10), 9, 141, 9, "1626", "4343"),
    10: (10, 10, (2, 3), 10, 251, 10, "1504", "4343"),
    11: (11, 11, (3, 4), 11, 252, 11, "1642", "4343"),
    12: (12, 12, (5, 6), 12, 254, 12, "1750", "4343"),
    13: (13, 13, (6, 7), 13, 255, 13, "1764", "4343"),
    14: (14, 14, (7, 8), 14, 256, 14, "1772", "4343"),
    15: (15, 15, (8, 9), 15, 257, 15, "1775", "4343"),
    16: (16, 16, (9, 10), 16, 258, 16, "1776", "4343"),
    17: (17, 17, (1, 4), 17, 469, 17, "1156", "4343"),
    18: (18, 18, (2, 5), 18, 470, 18, "1467", "4343"),
    19: (19, 19, (3, 6), 19, 471, 19, "1633", "4343"),
    20: (20, 20, (4, 7), 20, 472, 20, "1715", "4343"),
    21: (21, 21, (5, 8), 21, 473, 21, "1746", "4343"),
    22: (22, 22, (6, 9), 22, 474, 22, "1763", "4343"),
    23: (23, 23, (1, 3), 23, 509, 23, "1063", "4343"),
    24: (24, 24, (4, 6), 24, 512, 24, "1706", "4343"),
    25: (25, 25, (5, 7), 25, 513, 25, "1743", "4343"),
    26: (26, 26, (6, 8), 26, 514, 26, "1761", "4343"),
    27: (27, 27, (7, 9), 27, 515, 27, "1770", "4343"),
    28: (28, 28, (8, 10), 28, 516, 28, "1774", "4343"),
    29: (29, 29, (1, 6), 29, 859, 29, "1127", "4343"),
    30: (30, 30, (2, 7), 30, 860, 30, "1453", "4343"),
    31: (31, 31, (3, 8), 31, 861, 31, "1625", "4343"),
    32: (32, 32, (4, 9), 32, 862, 32, "1712", "4343"),
    33: (65, 33, (5, 10), 33, 863, 33, "1745", "4343"),
    34: (66, 34, (4, 10), 34, 950, 34, "1713", "4343"),
    35: (67, 35, (1, 7), 35, 947, 35, "1134", "4343"),
    36: (68, 36, (2, 8), 36, 948, 36, "1456", "4343"),
    37: (69, 37, (4, 10), 37, 950, 37, "1713", "4343"),
}


def generate_code_sequence_L1CA(prn: int) -> np.ndarray[np.int8]:
    """Generates L1CA PRN code for given PRN.

    Parameters
    ----------
    prn : int
        the PRN of the signal/satellite

    Returns
    -------
    output : ndarray of shape(1023,)
        the complete binary (0/1) code sequence
    """
    ps = CODE_PHASE_ASSIGNMENTS[prn][2]
    g1 = generate_mls(10, [2, 9], [9])
    g2 = generate_mls(10, [1, 2, 5, 7, 8, 9], [ps[0] - 1, ps[1] - 1])
    code_seq_01 = (g1 + g2) % 2
    return code_seq_01.astype(np.int8)


_CODE_SEQUENCES_GPS_L1CA = {}
def get_GPS_L1CA_code_sequence(prn: int) -> np.ndarray:
    '''
    Returns the code sequence corresponding to the given PRN
    '''
    if prn not in _CODE_SEQUENCES_GPS_L1CA:
        _CODE_SEQUENCES_GPS_L1CA[prn] = generate_code_sequence_L1CA(prn)
    return _CODE_SEQUENCES_GPS_L1CA[prn]

In [2]:
prn = 2
code_seq = get_GPS_L1CA_code_sequence(prn)
first_10_chips = CODE_PHASE_ASSIGNMENTS[prn][6]
print("First 10 chips of PRN", prn, "code sequence:", code_seq[:10])
print("First 10 chips from table:", bin(int(first_10_chips, base=8)))

First 10 chips of PRN 2 code sequence: [1 1 1 0 0 1 0 0 0 0]
First 10 chips from table: 0b1110010000


2. What are the largest and smallest possible GPS satellite velocity relative to a stationary observer on the ground? What are the largest and smallest possible GPS L1, L2, and L5 signal Doppler frequencies?  Where do they occur relative to an observer on the ground?  You should take into consideration of the Earth rotation.


The Doppler frequencies with largest magnitude for an observer on the ground occur when the satellite is near the horizon.  This is because, at this point, the satellite velocity vector is more in line with the line-of-sight (LOS) vector between the satellite and observer.  When the satellite is rising, the LOS doppler is at its largest positive value, and when the satellite is setting, the LOS doppler is at its largest negative value.  Meanwhile, when the satellite is at zenith, the LOS doppler is zero since the satellite velocity vector is perpendicular to the LOS vector.  Consider the following illustrations of these scenarios:

Rising             |  At Zenith      |  Setting
:-------------------------:|:-------------------------:|:-------------------------:
![](./gnss-sat-los-geom-1.png)  |  ![](./gnss-sat-los-geom-2.png)  |  ![](./gnss-sat-los-geom-3.png)

First, let us define some constants:

- $R_\text{Earth} = 6378.137 \text{ km}$: Radius of the Earth
- $R_\text{GPS} = 26560 \text{ km}$: Semi-major axis of the GPS satellite orbit
- $\omega_\text{Earth} = 7.2921151467 \times 10^{-5} \text{ rad/s}$: Angular velocity of the Earth
- $\mu_\text{Earth} = 3.986005 \times 10^{14} \text{ m}^3/\text{s}^2$: Gravitational constant for the Earth
- $f_{L1} = 1575.42 \text{ MHz}$: L1 carrier frequency
- $f_{L2} = 1227.60 \text{ MHz}$: L2 carrier frequency
- $f_{L5} = 1176.45 \text{ MHz}$: L5 carrier frequency


In [3]:
import numpy as np
speed_of_light = 299792458  # m/s

mu_Earth = 3.986005e14  # m^3/s^2
omega_Earth = 7.2921151467e-5  # rad/s
R_earth = 6371e3        # m
R_gps = 26560e3         # m
f_L1 = 1575.42e6        # Hz
f_L2 = 1227.60e6        # Hz
f_L5 = 1176.45e6        # Hz


Given that GPS satellites have a near-circular orbit, we can calculate their orbital speed using:

$$
v_\text{sat} = \sqrt{\frac{\mu_\text{Earth}}{R_\text{GPS} \times 10^3}} = 3874.5 \text{ m/s}
$$

In [4]:
v_GPS = np.sqrt(mu_Earth / R_gps)
print(f"GPS satellite speed: {v_GPS/1e3:.2f} km/s")

GPS satellite speed: 3.87 km/s


Next, we do some geometry to find the relationship between the satellite position, observer position, and the line-of-sight (LOS) component of the satellite velocity.  In the geometry below, we assume the observer is located at distance $R_\text{Earth}$ from the center of the Earth, and the satellite is located at distance $R_\text{GPS}$ from the center of the Earth.  The angle $\alpha$ is defined as the angle between the line connecting the center of the Earth to the satellite and the line connecting the observer to the satellite.  The green line represents the horizon plane for the observer.

<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
<img src="./gnss-orbit-geom.png" alt="GNSS Orbit Geometry" style="width: 400px;"/>
</div>


Using the law of sines, we can relate the satellite elevation angle $\theta_\text{el}$ to the angle $\alpha$:

$$
\frac{R_\text{Earth}}{\sin(\alpha)} = \frac{R_\text{GPS}}{\sin(90^\circ+\theta_\text{el})}
$$

which can be rearranged to solve for $\alpha$:

$$
\alpha = \arcsin\left(\frac{R_\text{Earth} }{R_\text{GPS}} \cdot \cos\theta_\text{el} \right)
$$


<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
<img src="./gnss-los-geom-closeup.png" alt="GNSS LOS Geometry Closeup" style="width: 200px;"/>
</div>

Meanwhile, the projection of the satellite velocity vector onto the LOS vector can be calculated using the inner (dot) product.  If we assume that the satellite velocity vector lies within the plane of the diagram, then this dot product simplifies to:

$$
\begin{align*}
v_\text{LOS} &= v_\text{GPS} \cdot \cos(90^\circ - \alpha) = v_\text{GPS} \cdot \sin(\alpha) \\
&= v_\text{GPS} \cdot \frac{R_\text{Earth} }{R_\text{GPS}} \cdot \cos\theta_\text{el}
\end{align*}
$$

This expression has the largest magnitude when $\theta_\text{el} = 0^\circ$ or $180^\circ$ (satellite at the horizon), and is zero when $\theta_\text{el} = 90^\circ$ (satellite at zenith).  Therefore, the maximum/minimum LOS velocity for this scenario is:

$$
v_\text{LOS,max} = v_\text{GPS} \cdot \frac{R_\text{Earth} }{R_\text{GPS}} \approx \pm 930 \text{ m/s}
$$

and the LOS velocity is zero when the satellite is at zenith.


In [5]:
v_LOS_max = v_GPS * R_earth / R_gps
print("Maximum LOS speed: ", v_LOS_max)  # m/s

Maximum LOS speed:  929.2539559287371



Note that if the satellite velocity vector is not in the plane of the diagram, then the LOS velocity will be reduced by a factor of $\cos(\beta)$, where $\beta$ is the angle between the satellite velocity vector and the plane of the diagram.  Therefore, the maximum LOS velocity calculated above represents an upper bound.  Also note that, when this is the case, the satellite does not pass through the zenith point for the observer.  This generally occurs when the satellite reaches its maximum elevation angle for its pass over the observer.

To complicate things, we can also consider the effect of the Earth's rotation on the LOS velocity.  We will only solve for the case where the observer is at the equator and Earth's rotation lies in the same plane as the satellite velocity vector, which is *NOT* the case for most satellite passes, including those of GPS satellites (whose orbital inclination is $55^\circ$).  For a more accurate description, its probably best to fully simulate the orbits and observer positions to assess min/max Doppler values.  However, for the sake of this problem, we will continue with this simplification.

<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
<img src="./gnss-orbit-geom-earth-rotation.png" alt="GNSS orbit geometry with Earth rotation" style="width: 200px;"/>
</div>

In [8]:
v_rot_max = omega_Earth * R_earth
print("Maximum speed at surface due to Earth rotation: ", v_rot_max)

Maximum speed at surface due to Earth rotation:  464.58065599625695


The velocity of the observer due to the Earth's rotation is perpendicular to its position vector and has a magnitude of:

$$
v_\text{obs} = \omega_\text{Earth} \cdot R_\text{Earth} \approx 465 \text{ m/s}
$$

The LOS component of this observer velocity can be calculated using the inner (dot) product:

$$
v_\text{obs,LOS} = v_\text{obs} \cdot \cos(\theta_\text{el})
$$

Therefore, when the satellite is rising/setting, the Earth rotation contributes at most $\pm 465$ m/s to the LOS velocity, and when the satellite is at zenith or highest elevevation, the Earth rotation contributes approximately zero to the LOS velocity.  Therefore, the maximum/minimum LOS velocity including the effect of Earth rotation is:

$$
\pm \left( 930 \text{ m/s} + 465 \text{ m/s} \right) = \pm 1395 \text{ m/s}
$$

The velocity of the observer due to the Earth's rotation is perpendicular to its position vector and has a magnitude of:

$$
v_\text{obs} = \omega_\text{Earth} \cdot R_\text{Earth} \approx 465 \text{ m/s}
$$

The LOS component of this observer velocity can be calculated using the inner (dot) product:

$$
v_\text{obs,LOS} = v_\text{obs} \cdot \cos(\theta_\text{el})
$$

Therefore, when the satellite is rising/setting, the Earth rotation contributes at most $\pm 465$ m/s to the LOS velocity, and when the satellite is at zenith or highest elevevation, the Earth rotation contributes approximately zero to the LOS velocity.  Therefore, the maximum/minimum LOS velocity including the effect of Earth rotation is:

$$
\pm \left( 930 \text{ m/s} + 465 \text{ m/s} \right) = \pm 1395 \text{ m/s}
$$

In [11]:
v_LOS_max_w_rot = v_LOS_max + v_rot_max

f_dopp_max_L1 = v_LOS_max_w_rot * f_L1 / speed_of_light
f_dopp_max_L2 = v_LOS_max_w_rot * f_L2 / speed_of_light
f_dopp_max_L5 = v_LOS_max_w_rot * f_L5 / speed_of_light

print(f"Maximum Doppler shift on L1: +/- {f_dopp_max_L1:.0f} Hz")
print(f"Maximum Doppler shift on L2: +/- {f_dopp_max_L2:.0f} Hz")
print(f"Maximum Doppler shift on L5: +/- {f_dopp_max_L5:.0f} Hz")

Maximum Doppler shift on L1: +/- 7325 Hz
Maximum Doppler shift on L2: +/- 5708 Hz
Maximum Doppler shift on L5: +/- 5470 Hz
